In [2]:
import pandas as pd
from sqlalchemy.dialects.mssql.information_schema import columns
from tpot import TPOTClassifier

df = pd.read_csv('extractedMimic.csv')  # Replace with the path to your CSV file


In [67]:
df.head()

,icustay_id,hadm_id,intime,outtime,dbsource,suspected_infection_time_poe,suspected_infection_time_poe_days,specimen_poe,positiveculture_poe,antibiotic_time_poe,...,glucose_min1,glucose_max1,glucose_mean,rrt,subject_id,hadm_id.1,icustay_id.1,urineoutput,colloid_bolus,crystalloid_bolus
0,205941,156324,28/5/2157 14:26:21,30/5/2157 14:18:24,metavision,28/5/2157 15:30:00,-0.044201,MRSA SCREEN,0,28/5/2157 00:00:00,...,40.0,202.0,87.250000,0,88883,156324,205941,0.0,NaN,250.0
1,252848,163315,29/7/2196 02:26:17,29/7/2196 12:02:39,metavision,29/7/2196 04:57:00,-0.104664,MRSA SCREEN,1,29/7/2196 00:00:00,...,182.0,231.0,206.500000,0,46154,163315,252848,0.0,NaN,250.0
2,237901,180937,14/2/2145 17:55:07,23/2/2145 12:43:43,metavision,14/2/2145 21:20:00,-0.142280,BLOOD CULTURE,0,15/2/2145 00:00:00,...,123.0,185.0,151.285714,1,42682,180937,237901,0.0,NaN,250.0
3,207491,143962,11/6/2159 12:47:02,14/6/2159 16:31:30,metavision,11/6/2159 12:11:00,0.025023,BLOOD CULTURE,0,11/6/2159 00:00:00,...,92.0,118.0,105.000000,0,45111,143962,207491,4.0,NaN,250.0
4,293063,118489,1/1/2135 17:28:33,2/1/2135 06:56:56,metavision,1/1/2135 15:55:00,0.064965,BLOOD CULTURE,0,1/1/2135 00:00:00,...,150.0,163.0,155.000000,0,56648,118489,293063,5.0,NaN,250.0


In [12]:
from sklearn.model_selection import train_test_split

# Step 1: Prepare the dataset for machine learning
# Drop the ICUSTAY_ID and ROW_ID columns, and separate features from labels
X = df[['urineoutput', 'lactate_min','Bun_mean'.lower(),'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()]]
# 'Urineoutput'.lower(), 'Lactate_min'.lower(), 'Bun_mean'.lower(), 'Sysbp_min'.lower(), 'Metastatic_cancer'.lower(), 'Inr_max'.lower(), 'Age'.lower(), 'Sodium_max'.lower(), 'Aniongap_max'.lower(), 'Creatinine_min'.lower(), 'Spo2_mean'.lower()
y = df['thirtyday_expire_flag']

# Step 2: Handle missing values (example: filling with median or mean)
# This can be adjusted based on your specific needs and data distribution
X.fillna(X.median(), inplace=True)  # Fill missing values with median

# Step 3: Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print shapes of the resulting datasets
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (3191, 11)
Testing set shape: (1368, 11)


C:\Users\beta\AppData\Local\Temp\ipykernel_123692\884630001.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)  # Fill missing values with median


In [13]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Train the XGBoost model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Step 2: Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Step 3: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print evaluation metrics
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


Model Accuracy: 83.99%
Confusion Matrix:
[[1043   65]
 [ 154  106]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      1108
           1       0.62      0.41      0.49       260

    accuracy                           0.84      1368
   macro avg       0.75      0.67      0.70      1368
weighted avg       0.82      0.84      0.83      1368



In [3]:
df.isnull().sum()

icustay_id              0
hadm_id                 0
intime                  0
outtime                 0
dbsource                0
                     ... 
hadm_id.1               0
icustay_id.1            0
urineoutput             0
colloid_bolus        4050
crystalloid_bolus    1194
Length: 106, dtype: int64

In [9]:
from tpot import TPOTClassifier, TPOTRegressor
t = TPOTClassifier(generations=5 ,random_state=42 , verbosity=2)

t.fit(X_train, y_train)



Optimization Progress:   0%|          | 0/600 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8414259152967263

Generation 2 - Current best internal CV score: 0.8414259152967263

Generation 3 - Current best internal CV score: 0.8414259152967263

Generation 4 - Current best internal CV score: 0.8414259152967263

Generation 5 - Current best internal CV score: 0.8423653730113176

Best pipeline: RandomForestClassifier(GaussianNB(input_matrix), bootstrap=True, criterion=gini, max_features=0.9000000000000001, min_samples_leaf=9, min_samples_split=11, n_estimators=100)


TPOTClassifier(generations=5, random_state=42, verbosity=2)

In [15]:
from sklearn.ensemble import RandomForestClassifier


# Step 4: Initialize and train the Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Step 6: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the evaluation results
print(f"Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.8501
Confusion Matrix:
[[1078   30]
 [ 175   85]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1108
           1       0.74      0.33      0.45       260

    accuracy                           0.85      1368
   macro avg       0.80      0.65      0.68      1368
weighted avg       0.84      0.85      0.83      1368



In [17]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Define the XGBoost classifier with initial hyperparameters
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Step 2: Define hyperparameters for tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # Controls the step size (lower = more conservative)
    'max_depth': [3, 5, 7],             # Depth of each tree (higher = more complex model)
    'n_estimators': [100, 200, 300],     # Number of boosting rounds (trees)
    'subsample': [0.7, 0.8, 1.0],       # Ratio of samples used for training each tree
    'colsample_bytree': [0.7, 0.8, 1.0] # Ratio of features used by each tree
}

# Step 3: Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Step 4: Best model from Grid Search
best_model = grid_search.best_estimator_

# Step 5: Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Step 6: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the best hyperparameters and evaluation metrics
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

# Step 7: Cross-validation on the entire training set
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy: {cv_scores.mean() * 100:.2f}%")


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Model Accuracy: 85.60%
Confusion Matrix:
[[1081   27]
 [ 170   90]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1108
           1       0.77      0.35      0.48       260

    accuracy                           0.86      1368
   macro avg       0.82      0.66      0.70      1368
weighted avg       0.85      0.86      0.83      1368

Cross-Validation Accuracy: 84.14%


In [34]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

# Define features and target variable
X = df[['urineoutput', 'lactate_min', 'bun_mean', 'sysbp_min', 
         'metastatic_cancer', 'inr_max', 'age', 'sodium_max', 
         'aniongap_max', 'creatinine_min', 'spo2_mean']]
y = df['thirtyday_expire_flag']

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Define hyperparameters for tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 150, 200],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Perform Grid Search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_search.fit(X, y)

# Best model from Grid Search
best_model = grid_search.best_estimator_

# Make predictions on the training data
y_pred = best_model.predict(X)
y_pred_proba = best_model.predict_proba(X)[:, 1]

# Evaluate the model's performance
auc_score = roc_auc_score(y, y_pred_proba)
conf_matrix = confusion_matrix(y, y_pred)
class_report = classification_report(y, y_pred, target_names=["No Expiration", "Expiration"], output_dict=True)

# Calculate 95% Confidence Interval using Bootstrapping
n_bootstraps = 1000
bootstrapped_scores = []
rng = np.random.RandomState(42)

for _ in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_proba), len(y_pred_proba))
    if len(np.unique(y.iloc[indices])) < 2:
        continue
    score = roc_auc_score(y.iloc[indices], y_pred_proba[indices])
    bootstrapped_scores.append(score)

# Confidence intervals
lower_bound = np.percentile(bootstrapped_scores, 2.5)
upper_bound = np.percentile(bootstrapped_scores, 97.5)

# Print the results
def print_classification_report(report):
    print("=== Model Performance Summary ===")
    print(f"Best Hyperparameters: {grid_search.best_params_}")
    print(f"Model AUC: {auc_score:.2f}")
    print(f"95% Confidence Interval for AUC: [{lower_bound:.2f}, {upper_bound:.2f}]")
    print("\nConfusion Matrix:")
    print(conf_matrix)
    print("\n=== Detailed Classification Report ===\n")

    for label, metrics in report.items():
        if label in ["No Expiration", "Expiration"]:
            print(f"Class: {label}")
            print(f"  Precision: {metrics['precision']:.2f}")
            print(f"  Recall: {metrics['recall']:.2f}")
            print(f"  F1-Score: {metrics['f1-score']:.2f}")
            print(f"  Support: {metrics['support']}")
            print()

    print(f"Overall Accuracy: {report['accuracy']:.2f}")
    print(f"Macro Average Precision: {report['macro avg']['precision']:.2f}")
    print(f"Macro Average Recall: {report['macro avg']['recall']:.2f}")
    print(f"Macro Average F1-Score: {report['macro avg']['f1-score']:.2f}")
    print(f"Weighted Average Precision: {report['weighted avg']['precision']:.2f}")
    print(f"Weighted Average Recall: {report['weighted avg']['recall']:.2f}")
    print(f"Weighted Average F1-Score: {report['weighted avg']['f1-score']:.2f}")

    print("\n=== Interpretation ===")
    print("The model shows good precision in predicting non-expiration cases, "
          "but recall for expiration cases indicates room for improvement.")
    print("Consider exploring class weighting or additional feature engineering to improve predictions for the minority class.")

print_classification_report(class_report)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
=== Model Performance Summary ===
Best Hyperparameters: {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.7}
Model AUC: 0.88
95% Confidence Interval for AUC: [0.87, 0.90]

Confusion Matrix:
[[3605   65]
 [ 518  371]]

=== Detailed Classification Report ===

Class: No Expiration
  Precision: 0.87
  Recall: 0.98
  F1-Score: 0.93
  Support: 3670.0

Class: Expiration
  Precision: 0.85
  Recall: 0.42
  F1-Score: 0.56
  Support: 889.0

Overall Accuracy: 0.87
Macro Average Precision: 0.86
Macro Average Recall: 0.70
Macro Average F1-Score: 0.74
Weighted Average Precision: 0.87
Weighted Average Recall: 0.87
Weighted Average F1-Score: 0.85

=== Interpretation ===
The model shows good precision in predicting non-expiration cases, but recall for expiration cases indicates room for improvement.
Consider exploring class weighting or additional feature engineering to improve predictions f